In [1]:
using VariableHeightMomentumControl
using RigidBodyDynamics
using RigidBodyDynamics.TreeDataStructure
using RigidBodyDynamics.OdeIntegrators
using StaticArrays

adding: /Users/twan/code/julia/RigidBodyDynamics/v0.5/DrakeVisualizer/src/lcmtypes to the python path


In [2]:
# TODO: grab everything off of github and cache
DRAKE_DISTRO = "$(ENV["HOME"])/code/drake-distro"
urdf = "$DRAKE_DISTRO/drake/examples/Valkyrie/urdf/urdf/valkyrie_A_sim_drake_one_neck_dof_wide_ankle_rom.urdf"
mechanism = RigidBodyDynamics.parse_urdf(Float64, urdf);

In [3]:
# Create visualizer
vis = parse_urdf(urdf, mechanism; package_path = ["$DRAKE_DISTRO/drake/examples"]);

In [4]:
# Add sole frame
ankleRoll = edge_to_parent_data(findfirst(v -> !isroot(v) && edge_to_parent_data(v).name == "leftAnkleRoll", tree(mechanism)))
soleFrame = CartesianFrame3D("l_foot_sole")
soleToFoot = Transform3D(soleFrame, ankleRoll.frameAfter, SVector(0.067, 0., -0.09))
add_body_fixed_frame!(mechanism, soleToFoot);

In [5]:
# Reroot at the left foot and remove fixed joints
pelvis = vertex_data(findfirst(v -> name(vertex_data(v)) == "pelvis", tree(mechanism)))
leftFoot = vertex_data(findfirst(v -> name(vertex_data(v)) == "leftFoot", tree(mechanism)))
leftFootToWorld = Joint("", Fixed{Float64}())
world = root_body(mechanism)
reattach!(mechanism, pelvis, 
    world, leftFootToWorld, Transform3D(leftFootToWorld.frameBefore, world.frame, -soleToFoot.trans), 
    leftFoot, Transform3D{Float64}(leftFoot.frame, leftFootToWorld.frameAfter))
remove_fixed_joints!(mechanism);

In [6]:
# Create controller
centroidalFrame = CartesianFrame3D("centroidal")
pelvisVertex = findfirst(v -> name(vertex_data(v)) == "pelvis", tree(mechanism))
stanceFootVertex = findfirst(v -> name(vertex_data(v)) == "world", tree(mechanism))
stanceLegPath = path(pelvisVertex, stanceFootVertex)
positionControlledJoints = collect(filter(j -> j ∉ stanceLegPath.edgeData, joints(mechanism)))
controller = VariableHeightMomentumController(mechanism, positionControlledJoints, centroidalFrame, soleFrame);

In [7]:
function passive_dynamics!(vd::AbstractArray, t, state)
    dynamics!(result, state)
    copy!(vd, result.v̇)
end

passive_dynamics! (generic function with 1 method)

In [8]:
function controlled_dynamics!(vd::AbstractArray, t, state)
    torques = control(controller, t, state)
    dynamics!(result, state, torques)
    copy!(vd, result.v̇)
end

controlled_dynamics! (generic function with 1 method)

In [9]:
# Simulation
state = MechanismState(Float64, mechanism)
result = DynamicsResult(Float64, mechanism)
integrator = MuntheKaasIntegrator(controlled_dynamics!, runge_kutta_4(Float64), DrakeVisualizerSink(vis, 1e-2));

In [10]:
ankleRoll = edge_to_parent_data(findfirst(v -> !isroot(v) && edge_to_parent_data(v).name == "leftAnkleRoll_flipped", tree(mechanism)))
knee = edge_to_parent_data(findfirst(v -> !isroot(v) && edge_to_parent_data(v).name == "leftKneePitch_flipped", tree(mechanism)))
hipPitch = edge_to_parent_data(findfirst(v -> !isroot(v) && edge_to_parent_data(v).name == "leftHipPitch_flipped", tree(mechanism)))
hipRoll = edge_to_parent_data(findfirst(v -> !isroot(v) && edge_to_parent_data(v).name == "leftHipRoll_flipped", tree(mechanism)))

Joint "leftHipRoll_flipped": Revolute joint with axis [-1.0,-0.0,-0.0]

In [12]:
xd = 1.0 + (1.4 - 1.0) * rand()
initialize_state!(state, xd, positionControlledJoints, ankleRoll, hipRoll, knee, hipPitch)
integrate(integrator, state, 1.5, 1e-3)
# control(controller, 0., state)

LoadError: LoadError: AssertionError: all(isnan(cop.v)) || isapprox(cop.v[1],0.0,atol=1.0e-8)
while loading In[12], in expression starting on line 3